# Evaluation of POLAR Dimensions

In [ ]:
import gensim
from numpy import linalg
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook as tqdm
import time
from random import shuffle
import sys
import nltk 
from nltk.corpus import wordnet 
import gc
from collections import defaultdict
import random
import json
import os
import pandas as pd
import pickle

import plotly
import numpy as np
import plotly.graph_objs as go
from sklearn.decomposition import PCA

from sklearn.cluster import DBSCAN

from functools import partial
import ipywidgets as widgets
from IPython.display import clear_output
from ipywidgets import IntProgress
from random import randint

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import multilabel_confusion_matrix

import torch

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile

from gensim.test.utils import datapath

Import the embedding matrix you want to use (Here you need to insert the embedding files one after another from your directory).

In [ ]:
#ignore common_df = pd.read_csv('/Users/stjepankusenic/POLAR_WEBE/data/processed/POLAR-GloVeWiki-bus-antonyms-common2.csv')
#import embedding for analysis, need to do analysis for every combiantion of pre-trained model and antonyms used 8 in total
company_df = pd.read_csv('/Users/stjepankusenic/POLAR_WEBE/data/processed/POLAR-GloVeWiki-bus-antonyms-inter.csv')
#Also input the embedding name and antonym set used here to get right file names in the end,
#bus=business antonym set we created, org=original antonyms used by POLAR paper
embedding_name = 'GloVeWiki_bus'

In [ ]:
#choose 10 companies that are in all embeddings for analysis
# exchange for common-10-companies-google file when using googlenews embeddings
with open("/Users/stjepankusenic/POLAR_WEBE/data/processed/common-10-companies", "rb") as fp:  
    b = pickle.load(fp)
    
analysis_df = company_df.loc[company_df['Unnamed: 0'].isin(b)]
df_names = analysis_df['Unnamed: 0']

Functions for creating the test setup are defined (Just execute).

In [ ]:
def get_important_pairs(df, number, name):
    #get the number of highest absolute values
    imp_row = df.loc[df['Unnamed: 0']==name]
    imp_row = imp_row.loc[:,imp_row.columns!='Unnamed: 0']
    imp_columns = imp_row.abs().values.argsort(1)[:, -number:][:, ::-1][0]
    #get the column names
    column_names=[]
    column_list=imp_row.columns.values.tolist()
    for i in imp_columns:
        nam=column_list[i]
        column_names.append(nam)
    value_list=[]
    for item in column_names:
        value_list.append(imp_row[item].values)
    ret_df=pd.DataFrame(column_names)
    ret_df.columns=['top_polar_dim']
    ret_df['top_value']=value_list
    imp_columns_down=imp_row.abs().values.argsort(1)[:, :number][:, ::-1][0]
    #display(imp_columns_down)
    down_column_names=[]
    down_value_list=[]
    for i in imp_columns_down:
        nam=column_list[i]
        down_column_names.append(nam)
        down_value_list.append(imp_row[nam].values)
    ret_df['down_polar_dim']=down_column_names
    ret_df['down_value']=down_value_list
    
    return ret_df    

In [ ]:
def create_polar_intruder_dataset(number,bus_df,name):
    company_data=get_important_pairs(bus_df,number,name)
    rand_list=[]
    pos_list=[]
    name_list=[]
    k = random.randint(0, number-1)
    for i in range(number):
        if i==k:
            rand_list.append(company_data['down_polar_dim'].loc[i])
            pos_list.append(k)
            name_list.append(name)
        else:
            rand_list.append(company_data['top_polar_dim'].loc[i])
            pos_list.append(k)
            name_list.append(name)
    company_data['random']= rand_list
    company_data['name'] = name_list
    company_data['position'] = pos_list
    return company_data

In [ ]:
#handle intruder creation outside
def create_polar_test_intruder_dataset(df):
    intruder_list=[]
    for bus in df:
        lst=create_polar_intruder_dataset(5,company_df,bus)
        intruder_list.append(lst)
    intruder_list= pd.concat(intruder_list, ignore_index=True)
    return intruder_list

In [ ]:
def polar_intrusion_test(df_intruder,name):

    max_count = df_intruder.shape[0]
    global i
    i = 0
    
    button_0 = widgets.Button(description = df_intruder['random'].loc[i])
    button_1 = widgets.Button(description = df_intruder['random'].loc[i+1])
    button_2 = widgets.Button(description = df_intruder['random'].loc[i+2])
    button_3 = widgets.Button(description = df_intruder['random'].loc[i+3])
    button_4 = widgets.Button(description = df_intruder['random'].loc[i+4])
    
    global chosen_positions
    chosen_positions=[]
    
    display("Polar Intrusion Text")
    
    f = IntProgress(min=0, max=max_count)    
    display(f)
    
    display(df_intruder['name'].loc[i])

    display(button_0)
    display(button_1)
    display(button_2)
    display(button_3)
    display(button_4)
    
    def btn_eventhandler(position, obj):
        global i 
        i += 5
        
        clear_output(wait=True)
            
        display("Polar Intrusion Text")
        display(f)
        f.value += 5
        
        global chosen_positions
        chosen_positions+=  5*[position]
                
        if i < max_count:
            
            display(df_intruder['name'].loc[i])

            button_0 = widgets.Button(description = df_intruder['random'].loc[i])
            button_1 = widgets.Button(description = df_intruder['random'].loc[i+1])
            button_2 = widgets.Button(description = df_intruder['random'].loc[i+2])
            button_3 = widgets.Button(description = df_intruder['random'].loc[i+3])
            button_4 = widgets.Button(description = df_intruder['random'].loc[i+4])
            
            display(button_0)
            display(button_1)
            display(button_2)
            display(button_3)
            display(button_4)
            
            button_0.on_click(partial(btn_eventhandler,0))
            button_1.on_click(partial(btn_eventhandler,1))
            button_2.on_click(partial(btn_eventhandler,2))
            button_3.on_click(partial(btn_eventhandler,3))
            button_4.on_click(partial(btn_eventhandler,4))
            
        else:
            print ("Thanks " + name + " you finished all the work!")
            #df_intruder['chosen_word'] = chosen_words
            df_intruder['chosen_position'] = chosen_positions
            df_intruder.to_csv("polar_intrusion_test_" + name +'_'+ embedding_name + "_results" + ".csv", index = False)
                
    button_0.on_click(partial(btn_eventhandler,0))
    button_1.on_click(partial(btn_eventhandler,1))
    button_2.on_click(partial(btn_eventhandler,2))
    button_3.on_click(partial(btn_eventhandler,3))
    button_4.on_click(partial(btn_eventhandler,4))

    
    return df_intruder

Execute the test:

In [ ]:
df_test =create_polar_test_intruder_dataset(df_names)
#change the name to your first name!
df_test1 =polar_intrusion_test(df_test,'first_name')

You don't need to do the evaluation. If you are finished with the test clear the output and insert new embedding file on top and start again.

In [ ]:
def test_evaluation(df1,df2,number):
    max_count= df1.shape[0]
    list1=[]
    list2=[]
    for i in range(number):
        list1.append(df1.loc[i*5])
        list2.append(df2.loc[i*5])
    df1= pd.DataFrame(list1)
    df2= pd.DataFrame(list2)
    df_both=pd.concat([df1,df2])
    f21=f1_score(df1['chosen_position'], df1['position'], average='weighted')
    print('F1 Score Coder 1:',f21)
    f22=f1_score(df2['chosen_position'], df2['position'], average='weighted')
    print('F1 Score Coder 2:',f22)
    print( )
    accuracy_score1=accuracy_score(df1['chosen_position'], df1['position'])
    print('Accuracy Score Coder 1:',accuracy_score1)
    accuracy_score2=accuracy_score(df2['chosen_position'], df2['position'])
    print('Accuracy Score Coder 2:',accuracy_score2)
    print( )
    precision_score1=precision_score(df1['chosen_position'], df1['position'], average='weighted',zero_division=1)
    print('Precision Score Coder 1:',precision_score1)
    precision_score2=precision_score(df2['chosen_position'], df2['position'], average='weighted',zero_division=1)
    print('Precision Score Coder 2:',precision_score2)
    print( )
    recall_score1=recall_score(df1['chosen_position'], df1['position'], average='weighted',zero_division=1)
    print('Recall Score Coder 1:',recall_score1)
    recall_score2=recall_score(df2['chosen_position'], df2['position'], average='weighted',zero_division=1)
    print('Recall Score Coder 2:',recall_score2)
    print( )
    
    kappa= cohen_kappa_score(df1['chosen_position'],df2['position'])
    print('Cohens Kappa for the Coders:',kappa)
    
test_evaluation(df_test1,df_test1,10)

## Ignore this part!

In [ ]:
def print_closest_words_to_business(name):#, n=5
    
    vec= company_df.loc[company_df['Unnamed: 0']==name]
    vec= vec.loc[:,vec.columns!='Unnamed: 0']
    vec= vec.values.tolist()
    vec= torch.tensor(vec[0]).unsqueeze(0)
    
    common_df1= common_df.loc[:,common_df.columns!='Unnamed: 0']
    index= common_df.shape[0]
    common_df1=common_df1.values.tolist()
    dists=[]
    for i in range(index):
        vect = torch.tensor(common_df1[i]).unsqueeze(0)
        dist = torch.cosine_similarity(vect, vec)
        dists.append(dist)
        
    lst=sorted(enumerate(dists), key=lambda x: x[1],reverse=True)
    return lst
    #for idx, difference in lst[:n]: 
        #print(common_df['Unnamed: 0'].loc[idx], difference.numpy())

In [ ]:
facebook_sim = print_closest_words_to_business('facebook')
for idx, difference in facebook_sim[:5]:
    print(common_df['Unnamed: 0'].loc[idx], difference.numpy())

In [ ]:
def get_important_pairs(df, number, name):
    #get the number of highest absolute values
    imp_row = df.loc[df['Unnamed: 0']==name]
    imp_row = imp_row.loc[:,imp_row.columns!='Unnamed: 0']
    imp_columns = imp_row.abs().values.argsort(1)[:, -number:][:, ::-1][0]
    #get the column names
    column_names=[]
    column_list=imp_row.columns.values.tolist()
    for i in imp_columns:
        nam=column_list[i]
        column_names.append(nam)
    value_list=[]
    for item in column_names:
        value_list.append(imp_row[item].values)
    ret_df=pd.DataFrame(column_names)
    ret_df.columns=['top_polar_dim']
    ret_df['top_value']=value_list
    imp_columns_down=imp_row.abs().values.argsort(1)[:, :number][:, ::-1][0]
    #display(imp_columns_down)
    down_column_names=[]
    down_value_list=[]
    for i in imp_columns_down:
        nam=column_list[i]
        down_column_names.append(nam)
        down_value_list.append(imp_row[nam].values)
    ret_df['down_polar_dim']=down_column_names
    ret_df['down_value']=down_value_list
    
    return ret_df   

In [ ]:
#function that selects intruder words, could be randomized
def create_intruder_dataset(number,all_words_df,name):
    company_data=print_closest_words_to_business(name)
    index_ranked = [i[0]for i in company_data]
    num_all = all_words_df.shape[0]
    intruder_list= []
    name_list=[]
    for i in index_ranked[num_all-number:]:
        intruder_list.append(all_words_df['Unnamed: 0'].loc[i])
        name_list.append(name)
    top_list =[]
    for i in index_ranked[:number]:
        top_list.append(all_words_df['Unnamed: 0'].loc[i])
    intruder_df = pd.DataFrame(intruder_list)
    intruder_df.columns = ['intruder']
    intruder_df['top'] = top_list
    rand_list=[]
    pos_list=[]
    k = random.randint(0, number-1)
    for i in range(number):
        if i==k:
            rand_list.append(intruder_list[i])
            pos_list.append(k)
        else:
            rand_list.append(top_list[i])
            pos_list.append(k)
    intruder_df['random']= rand_list
    intruder_df['name'] = name_list
    intruder_df['position'] = pos_list
    return intruder_df

In [ ]:
intruder_face= create_intruder_dataset(5,common_df,'facebook')
display(intruder_face)

In [ ]:
#function that creates test data sets for different coders
def create_test_dataset(df, test_number, name_1, name_2):
    length = df.shape[0]
    if 2*test_number > length:
        print('There are less business entites than test samples')
    df_shuffel = df.sample(frac=1).reset_index(drop=True)
    df_name1 = df_shuffel['Unnamed: 0'].loc[:test_number-1]
    df_name2 = df_shuffel['Unnamed: 0'].loc[test_number:2*test_number-1]
    return df_name1, df_name2

In [ ]:
df_stj, df_jkb = create_test_dataset(company_df,10,'Stjepan','Jakob')

In [ ]:
#handle intruder creation outside
def create_test_intruder_dataset(df):
    intruder_list=[]
    for bus in df:
        lst=create_intruder_dataset(5,common_df,bus)
        intruder_list.append(lst)
    intruder_list= pd.concat(intruder_list, ignore_index=True)
    return intruder_list

In [ ]:
#function that creates an interface for the intrusion test

def intrusion_test(df_intruder,name):

    max_count = df_intruder.shape[0]
    global i
    i = 0
    
    button_0 = widgets.Button(description = df_intruder['random'].loc[i])
    button_1 = widgets.Button(description = df_intruder['random'].loc[i+1])
    button_2 = widgets.Button(description = df_intruder['random'].loc[i+2])
    button_3 = widgets.Button(description = df_intruder['random'].loc[i+3])
    button_4 = widgets.Button(description = df_intruder['random'].loc[i+4])
    
    global chosen_positions
    chosen_positions=[]
    
    display("Word Intrusion Text")
    
    f = IntProgress(min=0, max=max_count)    
    display(f)
    
    display(df_intruder['name'].loc[i])

    display(button_0)
    display(button_1)
    display(button_2)
    display(button_3)
    display(button_4)
    
    def btn_eventhandler(position, obj):
        global i 
        i += 5
        
        clear_output(wait=True)
            
        display("Word Intrusion Text")
        display(f)
        f.value += 5
        
        global chosen_positions
        chosen_positions+=  5*[position]
                
        if i < max_count:
            
            display(df_intruder['name'].loc[i])

            button_0 = widgets.Button(description = df_intruder['random'].loc[i])
            button_1 = widgets.Button(description = df_intruder['random'].loc[i+1])
            button_2 = widgets.Button(description = df_intruder['random'].loc[i+2])
            button_3 = widgets.Button(description = df_intruder['random'].loc[i+3])
            button_4 = widgets.Button(description = df_intruder['random'].loc[i+4])
            
            display(button_0)
            display(button_1)
            display(button_2)
            display(button_3)
            display(button_4)
            
            button_0.on_click(partial(btn_eventhandler,0))
            button_1.on_click(partial(btn_eventhandler,1))
            button_2.on_click(partial(btn_eventhandler,2))
            button_3.on_click(partial(btn_eventhandler,3))
            button_4.on_click(partial(btn_eventhandler,4))
            
        else:
            print ("Thanks " + name + " you finished all the work!")
            #df_intruder['chosen_word'] = chosen_words
            df_intruder['chosen_position'] = chosen_positions
            df_intruder.to_csv("../data/processed/word_intrusion_test_" + name + "_results" + ".csv", index = False)
                
    button_0.on_click(partial(btn_eventhandler,0))
    button_1.on_click(partial(btn_eventhandler,1))
    button_2.on_click(partial(btn_eventhandler,2))
    button_3.on_click(partial(btn_eventhandler,3))
    button_4.on_click(partial(btn_eventhandler,4))

    
    return df_intruder
        

In [ ]:
test1= intrusion_test(df_stj,'Stjepan')